# Initialization

In [ ]:
import numpy as np
from pprint import pprint


from laboneq.contrib.example_helpers.generate_descriptor import generate_descriptor
#from laboneq.contrib.example_helpers.generate_device_setup import generate_device_setup
from laboneq.dsl.device import DeviceSetup




descriptor = generate_descriptor(
    #pqsc=[""], # 장비 여러개 사용시
    shfqc_6=["DEV12256"],
    number_data_qubits=1,
    multiplex=True,
    number_multiplex=1,
    include_cr_lines=True,
    include_ef_lines=True,
    get_zsync=False,  # Only set to True when using real device
    save = True,
    filename="MJ_KAIST"
)


#descriptor
setup = DeviceSetup.from_descriptor(yaml_text=descriptor, server_host="192.168.0.83")
#setup


In [2]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=True, do_emulation=True)
#session.disconnect()

[2025.09.03 17:31:20.368] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /Users/yalgaeahn/JSAHN/qubit-experiment/laboneq_output/log
[2025.09.03 17:31:20.369] INFO    VERSION: laboneq 2.58.0
[2025.09.03 17:31:20.370] INFO    Connecting to data server at 192.168.0.83:8004
[2025.09.03 17:31:20.370] INFO    Connected to Zurich Instruments LabOne Data Server version 25.07.0.507 at 192.168.0.83:8004
[2025.09.03 17:31:20.372] INFO    Configuring the device setup
[2025.09.03 17:31:20.373] INFO    The device setup is configured
[2025.09.03 17:31:20.369] INFO    VERSION: laboneq 2.58.0
[2025.09.03 17:31:20.370] INFO    Connecting to data server at 192.168.0.83:8004
[2025.09.03 17:31:20.370] INFO    Connected to Zurich Instruments LabOne Data Server version 25.07.0.507 at 192.168.0.83:8004
[2025.09.03 17:31:20.372] INFO    Configuring the device setup
[2025.09.03 17:31:20.373] INFO    The device setup is configured


In [3]:
from qpu_types.transmon import TransmonQubit, TransmonQubitParameters
from helper import load_qubit_parameters, save_qubit_parameters
from operations import TransmonOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform



#_loaded_qubits = load_qubit_parameters(filename="latest") 
#qubits = TransmonQubit.from_device_setup(setup) # Also can be done with load_qubit_parameters 
#qubits = list(_loaded_qubits.values())

qubits = []
for i in range(1):
    q = TransmonQubit.from_logical_signal_group(
        f"q{i}",
        setup.logical_signal_groups[f"q{i}"],
        parameters=TransmonQubitParameters()
    )
    qubits.append(q)



qpu = QPU(quantum_elements=qubits, quantum_operations=TransmonOperations())
#quantum_platform = QuantumPlatform(setup=setup, qpu=qpu)

# qpu.topology.add_edge("bus", "q16", "q17")
# qpu.topology.add_edge("bus", "q17", "q18")
# qpu.topology.add_edge("bus", "q18", "q19")
# qpu.topology.plot()




from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store") 
folder_store.activate()
#folder_store.deactivate()
workflow.logbook.LoggingStore().activate()
#workflow.logbook.LogbookStore().deactivate()


In [4]:
for q in qubits:
    q.parameters.ge_drive_pulse["sigma"] = 0.25
    q.parameters.readout_amplitude = 1.0
    q.parameters.reset_delay_length = 20e-6
    q.parameters.readout_range_out = -20
    q.parameters.readout_lo_frequency = 7.6e9
    # Set LO close to resonance to keep baseband frequency within NCO range
    q.parameters.drive_lo_frequency = 4.4e9
    q.parameters.drive_range = 10

qubits[0].parameters.resonance_frequency_ge = 4.4e9
qubits[0].parameters.resonance_frequency_ef = 4.2e9
qubits[0].parameters.readout_resonator_frequency = 7.6e9



# 1. Spectroscopy

### 1.1 Resonator spectroscopy with amplitude sweep

* CW spectroscopy can be done for wide range scan 
* amplitude, frequency 2D sweep 을 통해  power에 따른 broadening 및 SNR 경향 관찰

In [ ]:
from experiments import resonator_spectroscopy_amplitude

####################################################################################
q=qubits[0]
print(q.readout_parameters)
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-20e6,20e6,501)
amplitudes = np.linspace(0, 1.0, 21)
#####################################################################################
options = resonator_spectroscopy_amplitude.experiment_workflow.options()
#workflow.show_fields(options)
options.update(True)
options.use_cw(True)
options.do_analysis(False)
######################################################################################
# Build and run resonator_spectroscopy_amplitude wormflow
res_spec_amp = resonator_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)
res_spec_amp_result=res_spec_amp.run()


### 1.2 Resonator Spectroscopy
* CW not recommended 

In [26]:
from experiments import resonator_spectroscopy

####################################################################################
q=qubits[0]
print(q.readout_parameters())
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-20e6,20e6,501)
#####################################################################################
options = resonator_spectroscopy.experiment_workflow.options()
#workflow.show_fields(options)
options.update(False)
options.use_cw(True) #qop.measure 대신 
options.do_analysis(True)
######################################################################################
# Build and run resonator_spectroscopy_amplitude wormflow
res_spec = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    options=options
)
res_spec_result = res_spec.run()
res_spec_analysis_result = res_spec_result.tasks["analysis_workflow"]
pprint(res_spec_analysis_result.output)

('measure', {'amplitude': 1.0, 'length': 2e-06, 'pulse': {'function': 'const'}})
[2025.09.03 18:33:14.849] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.03 18:33:14.850] INFO      Workflow 'resonator_spectroscopy': execution started at 2025-09-03            
[2025.09.03 18:33:14.850] INFO      09:33:14.849143Z                                                              
[2025.09.03 18:33:14.851] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.03 18:33:14.862] INFO    Task 'temporary_qpu': started at 2025-09-03 09:33:14.862695Z
[2025.09.03 18:33:14.863] INFO    Task 'temporary_qpu': ended at 2025-09-03 09:33:14.863813Z
[2025.09.03 18:33:14.865] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-03 
[2025.09.03 18:33:14.866] INFO    09:33:14.865607Z
[2025.09.03 18:33:14.866] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-03 09:33:14.866462Z
[202

In [28]:
from laboneq.simple import show_pulse_sheet
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation

show_pulse_sheet(compiled_experiment=res_spec_result.tasks["compile_experiment"].output,name='test', interactive=True)
#plot_simulation(compiled_experiment=res_spec_result.tasks["compile_experiment"].output)

[2025.09.03 18:33:26.220] INFO    Recompiling the experiment due to missing extra information in the compiled experiment. Compile with `OUTPUT_EXTRAS=True` and `MAX_EVENTS_TO_PUBLISH=1000` to bypass this step with a small impact on the compilation time.
[2025.09.03 18:33:26.222] INFO    Resolved modulation type of oscillator 'q0_drive_ge_osc' on signal 'q0/drive' to HARDWARE
[2025.09.03 18:33:26.223] INFO    Resolved modulation type of oscillator 'q0_drive_ef_osc' on signal 'q0/drive_ef' to HARDWARE
[2025.09.03 18:33:26.224] INFO    Starting LabOne Q Compiler run...
[2025.09.03 18:33:26.336] INFO    Schedule completed. [0.111 s]
[2025.09.03 18:33:26.366] INFO    Code generation completed for all AWGs. [0.029 s]
[2025.09.03 18:33:26.367] INFO    Completed compilation step 1 of 1. [0.142 s]
[2025.09.03 18:33:26.369] INFO    Finished LabOne Q Compiler run.
[2025.09.03 18:33:26.369] INFO    Recompiling the experiment due to missing extra information in the compiled experiment. Compile with

### 2.1 Qubit Spectroscopy with Amplitude Sweep
* Current qubit spectroscopy modules do not support CW 
* Recommended to use long enough spectroscopy drive (pulsed) to see steady state signals

In [ ]:
from experiments import qubit_spectroscopy_amplitude

###############################################################
q = qubits[0]
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-20e6, 20e6, 41)
amplitudes = np.linspace(0.2, 1.0, 9)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")

pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
################################################################
options = qubit_spectroscopy_amplitude.experiment_workflow.options()
# workflow.show_fields(options)
###################################################################
qubit_spec_amp = qubit_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)

qubit_spec_amp_result = qubit_spec_amp.run()
qubit_spec_amp_result

### 2.2 Qubit Spectroscopy 


In [ ]:
from experiments import qubit_spectroscopy

###############################################################
q = qubits[0]
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-20e6, 20e6, 41)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
################################################################
options = qubit_spectroscopy.experiment_workflow.options()
# workflow.show_fields(options)
###################################################################
qubit_spec = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    options=options
)

qubit_spec_result = qubit_spec.run()
qubit_spec_result

# 2. Rabi

## 2.1 Rabi Amplitude 

In [ ]:
from laboneq_applications.experiments import amplitude_rabi
q = qubits[0]
#######################################################################
amplitudes = np.linspace(0,1.0,21)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = amplitude_rabi.experiment_workflow.options()
options.update(True)
options.use_cal_traces(False)
#print(workflow.show_fields(options))

###################################################################
rabi_amp = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    amplitudes=amplitudes,
    options=options
)

rabi_amp_result = rabi_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(rabi_amp_result.tasks['analysis_workflow'].output)

## 2.2 (optional) Rabi Amplitude Chevron 

In [ ]:
from experiments import amplitude_rabi_chevron

#######################################################################
q = qubits[0]
amplitudes = np.linspace(0,1.0,21)
frequencies = q.parameters.resonance_frequency_ge + np.linspace(-1e6, 1e6, 51)
print(f"DRIVE range (dBm) : {q.parameters.drive_range}, READOUT range (dBm) : {q.parameters.readout_range_out}")
#######################################################################
options = amplitude_rabi_chevron.experiment_workflow.options()

#print(workflow.show_fields(options))

###################################################################
rabi_amp_chev = amplitude_rabi_chevron.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options
)

rabi_amp_chev_result = rabi_amp_chev.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(rabi_amp_chev_result.tasks['analysis_workflow'].output)


## 2.3 (optional) Rabi Length

In [ ]:
from experiments import time_rabi

## 2.2 (optional) Rabi Length Chevron

In [ ]:
from experiments import time_rabi_chevron

# 3. Ramsey

In [ ]:
from experiments import ramsey

q = qubits[0]
#######################################################################
delays = np.linspace(0,1e-6,10)
detunings = 0.2e6
pprint(q.readout_parameters())
pprint(q.spectroscopy_parameters())
#######################################################################
options = ramsey.experiment_workflow.options()
options.update(False)
# Build and run Ramsey workflow (no explicit command table options required)
ramsey_wf = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    options=options
)
ramsey_result = ramsey_wf.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(ramsey_result.tasks['analysis_workflow'].output)

('measure', {'amplitude': 1.0, 'length': 2e-06, 'pulse': {'function': 'const'}})
('drive',
 {'amplitude': 1,
  'length': 5e-06,
  'pulse': {'can_compress': True, 'function': 'const'}})
[2025.09.03 17:37:56.784] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.03 17:37:56.785] INFO      Workflow 'ramsey': execution started at 2025-09-03 08:37:56.782839Z           
[2025.09.03 17:37:56.786] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.09.03 17:37:56.793] INFO    Task 'temporary_qpu': started at 2025-09-03 08:37:56.792904Z
[2025.09.03 17:37:56.794] INFO    Task 'temporary_qpu': ended at 2025-09-03 08:37:56.794502Z
[2025.09.03 17:37:56.798] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-09-03 
[2025.09.03 17:37:56.798] INFO    08:37:56.797880Z
[2025.09.03 17:37:56.799] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-09-03 08:37:56.799709Z
[2025.09.03 17:

# Basic Readout optimization
Here we optimize SNR via

* `readout_length`
* `readout-amplitude`

* `readout_integration_length`
* `readout_integration_delay`
* `readout_integration_weight`

In [12]:
from experiments import dispersive_shift

In [7]:
from experiments import time_traces

In [9]:
from experiments import signal_propagation_delay

# Save Qubit Parameters

In [ ]:
from helper import load_qubit_parameters, save_qubit_parameters
save_qubit_parameters(qubits={ qubit.uid : qubit for qubit in qubits}, save_folder="./qubit_parameters", filename="kaist")